# arXiv Bilingual Reader - TranslateGemma

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jimmyliao/trans-gemma/blob/main/arxiv-reader-colab.ipynb)

**專注於雙語閱讀學習的 arXiv 論文翻譯工具**

## 🎯 核心特色

- 📖 **雙語對照閱讀**：原文與翻譯並列，學習英文學術寫作
- 🧠 **智慧混合模式**：純文字頁面快速翻譯，含圖表頁面保留視覺上下文
- 📚 **分章節翻譯**：按論文結構（Abstract, Method, Experiments）組織
- 💡 **術語對照表**：自動提取專業術語，建立中英對照
- 🇹🇼 **繁體中文優化**：確保輸出台灣慣用的繁體中文（已修復官方 bug）

## 🎓 適合誰使用？

- ✅ 研究生學習閱讀英文論文
- ✅ 研究者深入研讀重要文獻
- ✅ 準備論文寫作/審稿的學者
- ✅ 想學習英文學術寫作的人

## ⚡ 快速開始

1. 執行 Cell 1-2（環境設置 + 認證）
2. 在 Cell 3 輸入 arXiv 論文 ID 和設定
3. 執行 Cell 4 開始翻譯
4. 享受雙語對照閱讀！

---

## 👤 About the Author

**Jimmy Liao** - AI Google Developer Expert (GDE), CTO/Co-Founder

- 🐦 Twitter: [@jimmyliao](https://twitter.com/jimmyliao)
- 💼 LinkedIn: [jimmyliao](https://linkedin.com/in/jimmyliao)
- 📝 Blog: [memo.jimmyliao.net](https://memo.jimmyliao.net)
- 🔗 GitHub: [jimmyliao/trans-gemma](https://github.com/jimmyliao/trans-gemma)

---

## 1️⃣ Setup: Clone Repository & Install Dependencies

In [ ]:
## 2️⃣ HuggingFace Authentication

**IMPORTANT:** TranslateGemma is a gated model. You need to:
1. Get a HuggingFace token from [https://huggingface.co/settings/tokens](https://huggingface.co/settings/tokens)
2. Accept model access at [https://huggingface.co/google/translategemma-4b-it](https://huggingface.co/google/translategemma-4b-it)

### 🔐 Configuration Methods

#### **If using Web Colab** (Browser):
- Click the 🔑 icon on left sidebar
- Add secret: `HF_TOKEN` = your token
- Run the cell below

#### **If using VS Code Colab Extension**:
Set the environment variable **before** opening VS Code:

```bash
# Option 1: Temporary (in terminal)
export HF_TOKEN="hf_xxxxxxxxxxxx"
code .

# Option 2: Permanent (in ~/.zshrc or ~/.bashrc)
echo 'export HF_TOKEN="hf_xxxxxxxxxxxx"' >> ~/.zshrc
source ~/.zshrc
```

Or the cell will prompt you to enter it manually.

from huggingface_hub import login
import os

def get_hf_token():
    """Smart HF Token retrieval (supports both Web Colab and VS Code)"""
    
    # Method 1: Try environment variables (works everywhere)
    token = os.getenv('HF_TOKEN') or os.getenv('HUGGING_FACE_HUB_TOKEN')
    if token:
        print("✅ HF_TOKEN loaded from environment variable")
        return token
    
    # Method 2: Try Colab Secrets (Web Colab only)
    try:
        from google.colab import userdata
        token = userdata.get('HF_TOKEN')
        print("✅ HF_TOKEN loaded from Colab Secrets (Web Colab)")
        return token
    except Exception:
        pass  # VS Code Colab Extension falls through here
    
    # Method 3: Manual input (fallback)
    print("\n" + "="*80)
    print("⚠️  HF_TOKEN Configuration Required")
    print("="*80)
    print("\n📝 Please enter your HuggingFace token:")
    print("   Get token: https://huggingface.co/settings/tokens")
    print("   Accept access: https://huggingface.co/google/translategemma-4b-it")
    print()
    
    token = input("HuggingFace Token: ").strip()
    
    if token:
        print("✅ HF_TOKEN set manually")
        return token
    else:
        raise ValueError("❌ HF_TOKEN is required to use TranslateGemma")

# Authenticate
try:
    HF_TOKEN = get_hf_token()
    os.environ['HF_TOKEN'] = HF_TOKEN  # Also set as env var for consistency
    login(token=HF_TOKEN)
    print("\n✅ Successfully authenticated with HuggingFace\n")
except Exception as e:
    print(f"\n❌ Authentication failed: {e}\n")
    raise

In [ ]:
from huggingface_hub import login
import os

try:
    # Try to load from Colab secrets (recommended)
    from google.colab import userdata
    HF_TOKEN = userdata.get('HF_TOKEN')
    os.environ['HF_TOKEN'] = HF_TOKEN
    login(token=HF_TOKEN)
    print("✅ Authenticated with HuggingFace using Colab secrets")
except Exception as e:
    print("⚠️  Could not load HF_TOKEN from Colab secrets")
    print("Please add HF_TOKEN to Colab Secrets (🔑 icon on left sidebar)")
    print(f"Error: {e}")
    
    # Fallback: Manual token input
    print("\n📝 Or enter your token manually:")
    HF_TOKEN = input("HuggingFace Token: ")
    os.environ['HF_TOKEN'] = HF_TOKEN
    login(token=HF_TOKEN)
    print("✅ Authenticated with HuggingFace")

## 3️⃣ Configuration: Bilingual Reading Settings

**配置你的論文和翻譯設定**

In [ ]:
# ============================================
# arXiv Paper Configuration
# ============================================

# arXiv Paper ID (from URL: https://arxiv.org/abs/XXXX.XXXXX)
ARXIV_ID = "2601.09012v2"  # TranslateGemma Technical Report

# ============================================
# Bilingual Reading Settings
# ============================================

# Enable bilingual mode (always recommended for learning)
BILINGUAL_MODE = True

# Target language
TARGET_LANG = "zh-TW"  # Traditional Chinese
SOURCE_LANG = "en"     # English

# ============================================
# Translation Sections (Page Ranges)
# ============================================

# Define which sections to translate
# Format: "section_name": (start_page, end_page)
SECTIONS = {
    "abstract": (1, 1),      # Abstract (Page 1)
    "method": (3, 5),        # Method section (Pages 3-5)
    "experiments": (7, 9),   # Experiments (Pages 7-9)
}

# ============================================
# Smart Mode Selection
# ============================================

# Pages that contain figures/tables (use image mode for these)
# Image mode is slower (40-50s) but preserves visual context
IMAGE_MODE_PAGES = [3, 5, 7]  # Pages with Figure 1, Table 1, etc.

# DPI for image mode (lower = faster, higher = better quality)
DPI = 96  # Options: 72 (fast), 96 (balanced), 150 (high quality)

# ============================================
# Display Settings
# ============================================

# Extract technical terms for glossary
EXTRACT_TERMS = True

# ============================================
# Summary
# ============================================

print("\n" + "="*80)
print("🎯 arXiv Bilingual Reader - Configuration")
print("="*80)
print(f"\n📄 Paper: arXiv:{ARXIV_ID}")
print(f"🌐 Translation: {SOURCE_LANG} → {TARGET_LANG}")
print(f"📖 Bilingual Mode: {'✅ Enabled' if BILINGUAL_MODE else '❌ Disabled'}")
print(f"\n📚 Sections to translate:")
for section, (start, end) in SECTIONS.items():
    pages = f"Page {start}" if start == end else f"Pages {start}-{end}"
    print(f"   • {section.capitalize()}: {pages}")
print(f"\n🖼️  Image mode pages: {IMAGE_MODE_PAGES}")
print(f"   (DPI: {DPI}, slower but preserves figures/tables)")
print(f"\n💡 Ready to translate! Execute Cell 4 to start.")
print("="*80 + "\n")

## 4️⃣ Translate: Smart Bilingual Mode

**執行智慧雙語翻譯**

- 📄 Text mode: 20-25s/page (pure text pages)
- 🖼️ Image mode: 40-50s/page (pages with figures/tables)

In [ ]:
import sys
import time
from pathlib import Path

# Add examples to path
sys.path.insert(0, 'examples')
sys.path.insert(0, 'examples/backends')

# Import backends
from transformers_backend import TransformersBackend
from transformers_multimodal_backend import TransformersMultimodalBackend

# Import utilities
import urllib.request
import fitz  # PyMuPDF
from PIL import Image
import re

# ============================================
# Helper Functions
# ============================================

def download_arxiv(arxiv_id):
    """Download arXiv PDF"""
    # Clean arXiv ID (remove version if present)
    clean_id = arxiv_id.split('v')[0] if 'v' in arxiv_id else arxiv_id
    
    url = f"https://arxiv.org/pdf/{clean_id}.pdf"
    filename = f"{arxiv_id}.pdf"
    
    print(f"📥 Downloading from arXiv: {arxiv_id}")
    print(f"   URL: {url}")
    
    urllib.request.urlretrieve(url, filename)
    
    # Check PDF info
    doc = fitz.open(filename)
    total_pages = len(doc)
    doc.close()
    
    print(f"✅ Downloaded: {filename} ({total_pages} pages)\n")
    return filename, total_pages

def extract_text_from_page(pdf_path, page_num):
    """Extract text from specific PDF page"""
    doc = fitz.open(pdf_path)
    page = doc[page_num - 1]  # 0-indexed
    text = page.get_text()
    doc.close()
    return text.strip()

def page_to_image(pdf_path, page_num, dpi=96):
    """Convert PDF page to image"""
    doc = fitz.open(pdf_path)
    page = doc[page_num - 1]  # 0-indexed
    pix = page.get_pixmap(dpi=dpi)
    img_data = pix.tobytes("ppm")
    img = Image.frombytes("RGB", [pix.width, pix.height], img_data)
    doc.close()
    return img

def display_bilingual(original, translation, page_num, mode="text", time_taken=0):
    """Display bilingual content in paragraph layout"""
    mode_icon = "📄" if mode == "text" else "🖼️"
    mode_text = "Text Mode" if mode == "text" else "Image Mode"
    
    print("\n" + "="*80)
    print(f"{mode_icon} Page {page_num} - {mode_text} (⏱️  {time_taken:.1f}s)")
    print("="*80)
    
    if BILINGUAL_MODE:
        # Bilingual layout (paragraph style)
        print("\n" + "─"*80)
        print("原文 (Original)")
        print("─"*80)
        # Truncate original if too long (show first 500 chars)
        orig_display = original[:500] + "..." if len(original) > 500 else original
        print(orig_display)
        
        print("\n" + "─"*80)
        print("翻譯 (Translation)")
        print("─"*80)
        print(translation)
        print("\n" + "="*80)
    else:
        # Translation only
        print(translation)
        print("\n" + "="*80)
        print("💡 Tip: Set BILINGUAL_MODE = True to see original text")
        print("="*80)

def extract_technical_terms(original, translation):
    """Extract technical terms (simple version)"""
    terms = {}
    
    # Pattern: Chinese (English)
    pattern = r'([\u4e00-\u9fff]+)\s*\(([a-zA-Z][a-zA-Z\s]+)\)'
    matches = re.findall(pattern, translation)
    
    for zh, en in matches:
        terms[en.strip()] = zh.strip()
    
    return terms

# ============================================
# Main Translation Flow
# ============================================

print("\n" + "#"*80)
print("# arXiv Bilingual Reader - Translation Started")
print("#"*80 + "\n")

# Download arXiv paper
pdf_path, total_pages = download_arxiv(ARXIV_ID)

# Load backends
print("🔄 Loading translation models...\n")

# Text backend (for pure text pages)
text_backend = TransformersBackend()
print("📄 Loading text backend...")
text_backend.load_model()
print("✅ Text backend ready\n")

# Multimodal backend (for image pages)
image_backend = TransformersMultimodalBackend()
print("🖼️  Loading multimodal backend...")
image_backend.load_model()
print("✅ Multimodal backend ready\n")

# Track all terms
all_terms = {}

# Translate each section
for section_name, (start_page, end_page) in SECTIONS.items():
    print("\n" + "#"*80)
    print(f"# Section: {section_name.upper()}")
    print(f"# Pages: {start_page}-{end_page}")
    print("#"*80 + "\n")
    
    for page_num in range(start_page, end_page + 1):
        # Check if this page should use image mode
        use_image_mode = page_num in IMAGE_MODE_PAGES
        
        if use_image_mode:
            print(f"🖼️  Translating page {page_num} with image mode (contains figures/tables)...")
            
            # Extract original text for display
            original_text = extract_text_from_page(pdf_path, page_num)
            
            # Convert to image and translate
            page_image = page_to_image(pdf_path, page_num, dpi=DPI)
            
            start_time = time.time()
            result = image_backend.translate_image(
                page_image,
                source_lang=SOURCE_LANG,
                target_lang=TARGET_LANG,
                stream=False  # Disable streaming for cleaner output
            )
            time_taken = time.time() - start_time
            
            translation = result['translation']
            
        else:
            print(f"📄 Translating page {page_num} with text mode (pure text)...")
            
            # Extract text
            original_text = extract_text_from_page(pdf_path, page_num)
            
            if not original_text:
                print("⚠️  Warning: No text found on this page, skipping...\n")
                continue
            
            # Translate
            start_time = time.time()
            result = text_backend.translate(
                original_text,
                source_lang=SOURCE_LANG,
                target_lang=TARGET_LANG
            )
            time_taken = time.time() - start_time
            
            translation = result['translation']
        
        # Display bilingual result
        mode = "image" if use_image_mode else "text"
        display_bilingual(original_text, translation, page_num, mode, time_taken)
        
        # Extract terms
        if EXTRACT_TERMS:
            terms = extract_technical_terms(original_text, translation)
            all_terms.update(terms)

# Display technical terms glossary
if EXTRACT_TERMS and all_terms:
    print("\n" + "#"*80)
    print("# 📚 Technical Terms Glossary")
    print("#"*80 + "\n")
    
    print("="*80)
    print("專業術語對照表")
    print("="*80)
    
    for en, zh in sorted(all_terms.items()):
        print(f"{en:40} → {zh}")
    
    print("\n" + "="*80)
    print(f"Total terms extracted: {len(all_terms)}")
    print("="*80)

print("\n" + "#"*80)
print("# ✅ Translation Complete!")
print("#"*80 + "\n")

print("💡 Tips:")
print("   • Scroll up to review the bilingual translations")
print("   • Check the technical terms glossary above")
print("   • Adjust IMAGE_MODE_PAGES in Cell 3 if needed")
print("   • Try different sections by modifying SECTIONS")

## 📝 Notes & Tips

### 🎯 How to Use This Notebook

1. **First Time Setup**:
   - Run Cell 1-2 to setup environment (takes ~2-3 minutes)
   - Add your HuggingFace token to Colab Secrets

2. **Configure Your Paper** (Cell 3):
   - Change `ARXIV_ID` to your target paper
   - Adjust `SECTIONS` to translate specific parts
   - Mark pages with figures/tables in `IMAGE_MODE_PAGES`

3. **Run Translation** (Cell 4):
   - Scroll through bilingual results
   - Check technical terms glossary at the end

### ⚡ Performance Tips

- **Text Mode**: 20-25s/page - Use for pure text pages
- **Image Mode**: 40-50s/page - Use for pages with figures/tables
- **DPI Setting**: Lower DPI (72-96) is faster, higher (150) preserves details

### 🖼️ When to Use Image Mode?

Use `IMAGE_MODE_PAGES` for pages containing:
- Figures (charts, diagrams, architecture)
- Tables (results, comparisons)
- Complex layouts (multi-column, mixed text/images)

Image mode preserves the visual context and relationship between text and graphics.

### 📚 Bilingual Reading Benefits

- **Learn Technical Terms**: See how concepts are expressed in English
- **Verify Translation**: Check if key terms are translated correctly
- **Academic Writing**: Learn English academic writing patterns
- **Research Quality**: Ensure you understand the original meaning

### 🔧 Customization

**To translate different sections**:
```python
SECTIONS = {
    "introduction": (2, 3),
    "related_work": (4, 5),
    "conclusion": (15, 16),
}
```

**To disable bilingual mode** (translation only):
```python
BILINGUAL_MODE = False
```

**To translate to other languages**:
```python
TARGET_LANG = "ja"     # Japanese
TARGET_LANG = "ko"     # Korean
TARGET_LANG = "zh-CN"  # Simplified Chinese
```

---

## 🔗 Links

- 📦 **GitHub Repository**: [jimmyliao/trans-gemma](https://github.com/jimmyliao/trans-gemma)
- 🤗 **TranslateGemma Model**: [HuggingFace](https://huggingface.co/google/translategemma-4b-it)
- 📄 **Technical Report**: [arXiv:2601.09012](https://arxiv.org/abs/2601.09012)
- 🌐 **Google Blog**: [TranslateGemma Announcement](https://blog.google/innovation-and-ai/technology/developers-tools/translategemma/)

---

## 📄 License

MIT License - Free for commercial and non-commercial use

---

**有問題或建議？** 歡迎到 [GitHub Issues](https://github.com/jimmyliao/trans-gemma/issues) 提出！